In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/CS5246/AITA')

Mounted at /content/drive


In [2]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import string

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

import spacy
nlp = spacy.load("en_core_web_sm")

# from sumy.parsers.plaintext import PlaintextParser
# from sumy.nlp.tokenizers import Tokenizer
# from sumy.summarizers.lex_rank import LexRankSummarizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB, ComplementNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
def read_article(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    return sentences

In [22]:
porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()


def make_sentences(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

def tokenize_and_stem(text):
    tokens = word_tokenize(text)
    stems = [porter_stemmer.stem(token) for token in tokens]
    lemmas = [wordnet_lemmatizer.lemmatize(token) for token in stems]
    return lemmas

def read_article(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    # Tokenize and remove stopwords
    if stopwords is None:
        stopwords = []
    sent1 = [word.lower() for word in word_tokenize(sent1) if word.lower() not in stopwords]
    sent2 = [word.lower() for word in word_tokenize(sent2) if word.lower() not in stopwords]

    # Create a set of all unique words in both sentences
    all_words = list(set(sent1 + sent2))

    # Create vectors to represent each sentence
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Count word occurrences
    for word in sent1:
        vector1[all_words.index(word)] += 1

    for word in sent2:
        vector2[all_words.index(word)] += 1

    # Calculate cosine similarity
    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stopwords):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    # Fill the similarity matrix with cosine similarity scores
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stopwords)

    return similarity_matrix

def generate_summary(text, top_n=5):
    # Load English stopwords
    stop_words = stopwords.words('english')

    # Tokenize the text into sentences
    sentences = read_article(text)

    # Generate the sentence similarity matrix
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Convert the similarity matrix into a graph
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)

    # Apply the PageRank algorithm to rank the sentences
    scores = nx.pagerank(sentence_similarity_graph, max_iter=200)

    # Sort the sentences by their scores
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

    # Extract the top sentences as the summary
    summary = [sentence for score, sentence in ranked_sentences[:top_n]]

    while len(summary) < top_n:
      summary.append('')

    return summary

### preprocessing

In [6]:
part1 = pd.read_csv('submissions_part1.csv', parse_dates=[0])
part2 = pd.read_csv('submissions_part2.csv', parse_dates=[0])
df = pd.concat([part1, part2])
df

,created_time,title,score,id,content,AITA,upvotes
0,2021-07-03 12:36:22,AITA for telling my wife the lock on my daught...,81014,ocx94s,My brother in-law (Sammy) lost his home shortl...,Not the A-hole,81014
1,2020-05-26 23:35:45,AITA For suing my girlfriend after she had my ...,70809,gr8bp3,I'll try to keep this short. I had a [1967 Imp...,Not the A-hole,70809
2,2019-12-03 17:07:53,AITA for pretending to get fired when customer...,63520,e5k3z2,I am a high schooler with a weekend job at a c...,Not the A-hole,63520
3,2022-12-26 12:44:43,"AITA for bringing up my brother's ""premature"" ...",60429,zvmflw,I am a nurse practitioner and I am the primary...,Not the A-hole,60429
4,2022-08-26 20:25:55,AITA for not taking down my video that was a g...,55748,wyjbjs,I have a sister that’s 6 years older than me. ...,Not the A-hole,55748
...,...,...,...,...,...,...,...
5956,2021-09-14 14:50:35,AITA for not letting my brother's girlfriend m...,5256,po4alt,I'm the middle child of 3 siblings. There's my...,Not the A-hole,5256
5957,2023-02-27 06:40:02,AITA for telling my husband I want his sisters...,2271,11d4zrj,"Please let me know if I’m in the wrong, becaus...",Not the A-hole,2271
5958,2023-09-19 22:29:59,AITA for telling my wife her sister can’t help...,2060,16n4e8z,My wife Dee (28F) and I (27M) recently had our...,Asshole,2060
5959,2023-12-07 17:55:08,AITA for telling my daughter (who is facing ho...,10580,18d19q5,**UPDATE:** *I do believe because of my anger ...,Not the A-hole,10580


In [16]:
df['flag'] = df['AITA'].apply(lambda x : 1 if x == "Asshole" else 0)
df.head()

,created_time,title,score,id,content,AITA,upvotes,flag
0,2021-07-03 12:36:22,AITA for telling my wife the lock on my daught...,81014,ocx94s,My brother in-law (Sammy) lost his home shortl...,Not the A-hole,81014,0
1,2020-05-26 23:35:45,AITA For suing my girlfriend after she had my ...,70809,gr8bp3,I'll try to keep this short. I had a [1967 Imp...,Not the A-hole,70809,0
2,2019-12-03 17:07:53,AITA for pretending to get fired when customer...,63520,e5k3z2,I am a high schooler with a weekend job at a c...,Not the A-hole,63520,0
3,2022-12-26 12:44:43,"AITA for bringing up my brother's ""premature"" ...",60429,zvmflw,I am a nurse practitioner and I am the primary...,Not the A-hole,60429,0
4,2022-08-26 20:25:55,AITA for not taking down my video that was a g...,55748,wyjbjs,I have a sister that’s 6 years older than me. ...,Not the A-hole,55748,0


In [17]:
df['flag'].value_counts()

0    10141
1     1782
Name: flag, dtype: int64

In [39]:
from sklearn.utils import resample

majority_df = df.query('flag==0')
downsampled_majority_df = resample(majority_df, replace=False, n_samples=1782)
balanced_df = pd.concat([df.query('flag==1'), downsampled_majority_df])
balanced_df = balanced_df.reset_index().drop('index', axis = 1)
balanced_df

,created_time,title,score,id,content,AITA,upvotes,flag
0,2019-07-16 15:41:11,AITA for telling cashier that wasn’t the girls...,41135,cdypge,\nThrowaway because husband told me I was TA a...,Asshole,41135,1
1,2019-06-02 14:10:33,AITA For Firing An Employee After His Parents ...,40289,bvxqh8,I'm the VP of Sales at a software company and ...,Asshole,40289,1
2,2020-07-15 14:48:00,AITA for not saying anything about the underwear?,40067,hrowir,My 20 year old daughter and her fiancé are cur...,Asshole,40067,1
3,2019-04-29 16:35:09,WIBTA if I ask my pregnant wife to move out be...,38263,birkjn,My wife is pregnant with our daughter. Initial...,Asshole,38263,1
4,2020-06-10 12:42:51,AITA for telling my stepdaughter to stop using...,37660,h0a45w,I have been living with my new wife and stepda...,Asshole,37660,1
...,...,...,...,...,...,...,...,...
3559,2022-08-25 12:12:35,AITA for asking to be moved to a different cla...,2817,wxclbx,"I, 18f, had a lot of problems last semester w...",Not the A-hole,2817,0
3560,2023-12-14 04:46:26,AITA or maybe I'm not,3,18i0pa7,"Little back story, almost 6 years ago, I was i...",Not the A-hole,3,0
3561,2020-08-28 10:58:33,AITA for asking my husband to turn down his dr...,35326,ii4zpu,"\nI’m going to be vague for privacy reasons, s...",Not the A-hole,35326,0
3562,2023-11-11 18:55:59,AITA for not wanting to go to my boyfriends fa...,10,17t123r,This is my first time writing about this incid...,Not the A-hole,10,0


In [40]:
sent1 = []
sent2 = []
sent3 = []
sent4 = []
sent5 = []
index = 0

for i, content in enumerate(balanced_df['content']):
  array_summary = generate_summary(content)
  sent1.append(array_summary[0])
  sent2.append(array_summary[1])
  sent3.append(array_summary[2])
  sent4.append(array_summary[3])
  sent5.append(array_summary[4])
  index = i

balanced_df['sent1'] = sent1
balanced_df['sent2'] = sent2
balanced_df['sent3'] = sent3
balanced_df['sent4'] = sent4
balanced_df['sent5'] = sent5


In [42]:
balanced_df.head()

,created_time,title,score,id,content,AITA,upvotes,flag,sent1,sent2,sent3,sent4,sent5
0,2019-07-16 15:41:11,AITA for telling cashier that wasn’t the girls...,41135,cdypge,\nThrowaway because husband told me I was TA a...,Asshole,41135,1,I tell cashier that isn’t her card but her fat...,Now that I’ve pointed out it wasn’t hers cashi...,I’ve told my son multiple times he’s never all...,Girl says it’s not fraud because she has permi...,Girl tells cashier her father gave her the car...
1,2019-06-02 14:10:33,AITA For Firing An Employee After His Parents ...,40289,bvxqh8,I'm the VP of Sales at a software company and ...,Asshole,40289,1,And I reached out to him to see how he's holdi...,He's a great employee and we were thinking abo...,He's staying at his gf's place and she's suppo...,"He's super unmotivated, not cold calling, out ...",I offered to get my head hunter friend in touc...
2,2020-07-15 14:48:00,AITA for not saying anything about the underwear?,40067,hrowir,My 20 year old daughter and her fiancé are cur...,Asshole,40067,1,Her fiancé said we’re fucked and left the hous...,I guess a pair of my wife’s underwear got in w...,My wife asked if I didn’t realize they were he...,I love my daughter but she is very difficult a...,My daughter burst into tears and won’t talk to...
3,2019-04-29 16:35:09,WIBTA if I ask my pregnant wife to move out be...,38263,birkjn,My wife is pregnant with our daughter. Initial...,Asshole,38263,1,And I passed.,When I asked her this she told me that she was...,I'm really pissed.,"Well, I made chicken sandwiches.","But then, she starts acting like a nut job."
4,2020-06-10 12:42:51,AITA for telling my stepdaughter to stop using...,37660,h0a45w,I have been living with my new wife and stepda...,Asshole,37660,1,My stepdaughters father isn’t present in her l...,Both of them have told me to stop being so sil...,My sons shouldn’t be subjected to her unhygien...,"She’s 19, almost 20, and I have three sons age...",I told her my sons were uncomfortable and aske...


In [51]:
balanced_df.drop('Unnamed: 0', axis = 1).to_csv('sampled_data.csv', index=False)

### cleaning and vectorizing

In [52]:
balanced_df = pd.read_csv('sampled_data.csv')

In [57]:
balanced_df = balanced_df.fillna('')

In [74]:
balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3564 entries, 0 to 3563
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   created_time  3564 non-null   object
 1   title         3564 non-null   object
 2   score         3564 non-null   int64 
 3   id            3564 non-null   object
 4   content       3564 non-null   object
 5   AITA          3564 non-null   object
 6   upvotes       3564 non-null   int64 
 7   flag          3564 non-null   int64 
 8   sent1         3564 non-null   object
 9   sent2         3564 non-null   object
 10  sent3         3564 non-null   object
 11  sent4         3564 non-null   object
 12  sent5         3564 non-null   object
dtypes: int64(3), object(10)
memory usage: 362.1+ KB


In [84]:
X_train, X_test, y_train, y_test = train_test_split(balanced_df[['sent1', 'sent2', 'sent3', 'sent4', 'sent5']], balanced_df['flag'], test_size=0.2, random_state=42)

In [87]:
def flatten(data):
    results = []
    for index, row in data.iterrows():
      results.extend(row)
    return results

In [88]:
X_train_flattened = flatten(X_train)
X_test_flattened = flatten(X_test)

In [101]:
# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer(max_features=10000)

# Fit the vectorizer to the data and transform the flattened samples into TF-IDF vectors
vectorizer.fit(X_train_flattened + X_test_flattened)
X_train_vector = vectorizer.transform(X_train_flattened)
X_test_vector = vectorizer.transform(X_test_flattened)

In [102]:
#Repeat each label 5 times to match the new flattened data
y_train_flattened = np.repeat(y_train, 5)
y_test_flattened = np.repeat(y_test, 5)

### models

In [103]:
#Stage 1 Logistic Regression
stage_1 = LogisticRegression(solver="saga")
stage_1.fit(X_train_vector, y_train_flattened)
X_train_intermediate_results = stage_1.predict(X_train_vector)
X_test_intermediate_results = stage_1.predict(X_test_vector)

In [104]:
#reshape to push into stage 2 logistic regression
X_train_intermediate_input = X_train_intermediate_results.reshape(int(len(X_train_intermediate_results)/5), 5)
X_test_intermediate_input = X_test_intermediate_results.reshape(int(len(X_test_intermediate_results)/5), 5)
print(X_train_intermediate_input.shape)
print(X_test_intermediate_input.shape)

(2851, 5)
(713, 5)


In [105]:
test = [round(np.mean(x)) for x in X_train_intermediate_input]
metrics.f1_score(y_train, test)

0.8475565280816921

In [106]:
test = [round(np.mean(x)) for x in X_test_intermediate_input]
metrics.f1_score(y_test, test)

0.5591098748261474

In [107]:
#Stage 2 Logistic Regression
stage_2 = LogisticRegression(solver="saga")
stage_2.fit(X_train_intermediate_input, y_train)
y_pred = stage_2.predict(X_test_intermediate_input)
metrics.f1_score(y_test, y_pred)

0.5591098748261474

In [83]:
#Stage 2 Logistic Regression
cross_val_scores = cross_val_score(LogisticRegression(solver="saga"), reshaped_arr, balanced_df['flag'], cv=10, scoring="f1")
print(f"cross-validation scores:", cross_val_scores)
print(f"average cross-validation score:", cross_val_scores.mean())

cross-validation scores: [0.81073446 0.82285714 0.82633053 0.87272727 0.82808023]
average cross-validation score: 0.8321459280600996


### workings

In [ ]:
#Various NB
cross_val_scores = cross_val_score(BernoulliNB(), X_train, y_train, cv=10, scoring="f1")
print(f"cross-validation scores:", cross_val_scores)
print(f"average cross-validation score:", cross_val_scores.mean())

NB_model = BernoulliNB()
NB_model.fit(X_train, y_train)
y_predict = NB_model.predict(X_test)
metrics.f1_score(y_test, y_predict)

cross-validation scores: [0.49716981 0.59329321 0.60054595 0.69833729 0.78442894 0.58047722
 0.44060914 0.46987952 0.56780402 0.70621075]
average cross-validation score: 0.5938755849977653


0.5836909871244635

In [ ]:
#Various NB
cross_val_scores = cross_val_score(ComplementNB(), X_train, y_train, cv=10, scoring="f1")
print(f"cross-validation scores:", cross_val_scores)
print(f"average cross-validation score:", cross_val_scores.mean())

NB_model = ComplementNB()
NB_model.fit(X_train, y_train)
y_predict = NB_model.predict(X_test)
metrics.f1_score(y_test, y_predict)

cross-validation scores: [0.58333333 0.61425061 0.63144548 0.72748815 0.74615673 0.58823529
 0.50691656 0.538218   0.64491525 0.73145009]
average cross-validation score: 0.6312409505781528


0.6205479452054794

In [ ]:
# Initialize TfidfVectorizer, MinMaxScaler, and Logistic Regression
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize_and_stem, token_pattern=None)
logistic_regression = LogisticRegression()

# Create a pipeline with TfidfVectorizer, Logistic Regression
pipeline = Pipeline([
    ('tfidf', tfidf_vectorizer),
    ('logistic', logistic_regression)
])

# Define hyperparameters grid for logistic regression
param_grid = {
    'tfidf__max_features': [5000, 6000, 7000], #9000, 10000],
    # 'tfidf__min_df': [0.0009, 0.0005, 0.0002, 1],
    #'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)], #
    'logistic__C': [0.1, 1, 10],  # Regularization parameter
    'logistic__penalty': ['l1', 'l2', 'elasticnet'],
    'logistic__fit_intercept': (True, False),
    'logistic__solver': ['saga'] #('newton-cg', 'sag', 'saga')
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=10, n_jobs=-1, scoring="f1")

# Train the grid search on the training data
grid_search.fit(justice_train['scenario'], justice_train['label'])

# Evaluate the best model on the test set
# best_model = grid_search.best_estimator_
# test_accuracy = best_model.score(X_test, y_test)
print("Best model test accuracy:", grid_search.best_score_)
print("Best hyperparameters:", grid_search.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
180 fits failed out of a total of 540.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1291, in fit
    fold_coefs_ = Parallel(n_jobs=self.n_jobs, verbose=self.ver

Best model test accuracy: 0.6435630019431773
Best hyperparameters: {'logistic__C': 1, 'logistic__fit_intercept': False, 'logistic__penalty': 'l1', 'logistic__solver': 'saga', 'tfidf__max_features': 5000}
